In [1]:
import os
import re
import cv2
import glob
import shutil
import random
import numpy as np
import pandas as pd
from keras import backend as K
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img


TRAIN_DIR = '../input/train-scene classification/train/'
TEST_DIR = '../input/test_WyRytb0.csv/'
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]

y_test = pd.read_csv("../input/test_WyRytb0.csv")
y_train = pd.read_csv("../input/train-scene classification/train.csv")
y_train = y_train['label']
from keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, 6)

train_images_list = []
test_images_list = []
a = list(y_test['image_name'])
for i in train_images:
    if str(i[42:]) not in a:
        train_images_list.append(i)
    if str(i[42:]) in a:
        test_images_list.append(i)
        
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

train_images_list.sort(key=natural_keys)
test_images_list.sort(key=natural_keys)

def prepare_data(list_of_images):
    x = [] 
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (150,150), interpolation=cv2.INTER_CUBIC))     
    return x
X_train = prepare_data(train_images_list)
X_test = prepare_data(test_images_list)



Using TensorFlow backend.


In [2]:
X_train_prime, X_val, y_train_prime, y_val = train_test_split(X_train, y_train_categorical, random_state = 6, train_size = .75)
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True)
train_generator = train_datagen.flow(np.array(X_train_prime), y_train_prime, batch_size = 16, shuffle = True)
validation_generator = val_datagen.flow(np.array(X_val), y_val, batch_size = 16, shuffle = True)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
from keras.applications.vgg19 import VGG19
vgg_conv = VGG19(weights = 'imagenet',include_top = False,input_shape = (150,150,3))

import tensorflow as tf
print(tf.__version__)

for layer in vgg_conv.layers[:8]:
    layer.trainable = True

for layer in vgg_conv.layers:
    print(layer,layer.trainable)
    
    
from keras import models,layers,optimizers
model = models.Sequential()
model.add(vgg_conv)
model.add(layers.Flatten())
model.add(layers.Dense(1024,  activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(6,activation = 'softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',optimizer = optimizers.Adam(lr=1e-6),metrics=['acc'])

80142336/80134624 [==============================] - 1s 0us/step
1.12.0
<keras.engine.input_layer.InputLayer object at 0x7f019f46b780> True
<keras.layers.convolutional.Conv2D object at 0x7f019f46bd30> True
<keras.layers.convolutional.Conv2D object at 0x7f019f46bba8> True
<keras.layers.pooling.MaxPooling2D object at 0x7f019f49bb38> True
<keras.layers.convolutional.Conv2D object at 0x7f019f49b668> True
<keras.layers.convolutional.Conv2D object at 0x7f019f3be0f0> True
<keras.layers.pooling.MaxPooling2D object at 0x7f019f583400> True
<keras.layers.convolutional.Conv2D object at 0x7f019f583dd8> True
<keras.layers.convolutional.Conv2D object at 0x7f019f5a5240> True
<keras.layers.convolutional.Conv2D object at 0x7f019f4605f8> True
<keras.layers.convolutional.Conv2D object at 0x7f019f450400> True
<keras.layers.pooling.MaxPooling2D object at 0x7f019f4e6358> True
<keras.layers.convolutional.Conv2D object at 0x7f019f4e6c88> True
<keras.layers.convolutional.Conv2D object at 0x7f019f4dc128> True
<k

In [4]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch = 1024,
    epochs = 5,
    shuffle = True,
    validation_data = validation_generator,
    validation_steps = 512,
    use_multiprocessing=True
)


Epoch 1/5
1024/1024 [==============================] - 403s 394ms/step - loss: 0.8481 - acc: 0.6788 - val_loss: 0.4290 - val_acc: 0.8425
Epoch 2/5
1024/1024 [==============================] - 393s 384ms/step - loss: 0.4271 - acc: 0.8453 - val_loss: 0.3394 - val_acc: 0.8791
Epoch 3/5
1024/1024 [==============================] - 392s 383ms/step - loss: 0.3457 - acc: 0.8796 - val_loss: 0.2883 - val_acc: 0.8958
Epoch 4/5
1024/1024 [==============================] - 393s 384ms/step - loss: 0.3031 - acc: 0.8886 - val_loss: 0.2627 - val_acc: 0.9052
Epoch 5/5
1024/1024 [==============================] - 393s 383ms/step - loss: 0.2724 - acc: 0.9053 - val_loss: 0.2541 - val_acc: 0.9075


In [5]:
X_test = np.array(X_test)/255.0
prediction_probabilities = model.predict(X_test)
counter = range(1, len(test_images_list) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
cols = ['label']
for i, j in enumerate(solution['label']):
    if(i%100 == 0):
        print(i+1)
    solution['label'][i] = np.argmax(j)
y_test['label'] = solution['label']
y_test.to_csv("Scene2.csv", index = False)

1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901
7001
7101
7201
7301
